**Part 4**

GRPO Loss

* Terrible Loss
* Gives One Sided Predictions

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
from tqdm import tqdm

In [ ]:
# Define the neural network
class LogicNet(nn.Module):
    def __init__(self):
        super(LogicNet, self).__init__()
        self.fc1 = nn.Linear(2, 4)  # Input layer -> Hidden Layer
        self.fc2 = nn.Linear(4, 1)  # Hidden Layer -> Output Layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        logits = self.fc2(x)
        return logits

    def get_action_and_or_log_prob(self, state, action=None):
        """Helper method to get action and its log_prob from logits"""
        logits = self.forward(state)    # Get the logits from a forward pass of the Policy Network
        # For a binary output (0 or 1), Bernoulli distribution is appropriate
        probs = torch.distributions.Bernoulli(logits=logits)

        if action is None:
            sampled_action = probs.sample() # Sample action based on current probabilities (returns 0 or 1)
            log_prob = probs.log_prob(sampled_action)   # Calculate the log of the probability the sampled action is chosen
            return sampled_action, log_prob
        else:
            log_prob = probs.log_prob(action)       # Returns the log of the probability the action is chosen
            return log_prob


In [25]:
# Define the environment
class LogicGateEnv:
    def __init__(self, gate="AND"):
        self.gate = gate
        self.data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
        self.targets = self.get_targets(gate)

    def get_targets(self, gate:str):
        if gate == "AND":
            return torch.tensor([[0], [0], [0], [1]], dtype=torch.float32)
        elif gate == "OR":
            return torch.tensor([[0], [1], [1], [1]], dtype=torch.float32)
        elif gate == "XOR":
            return torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)
        elif gate == "XNOR":
            return torch.tensor([[1], [0], [0], [1]], dtype=torch.float32)

    def step(self, input_idx: int, prediction):
        target = self.targets[input_idx]
        # Take the mean squared error
        # print(f"prediction: {prediction} || target: {target}")
        error = (prediction - target).pow(2).mean().item()
        reward = 1.0 - error
        return reward

In [ ]:
# Training loop
def train_logic_gate(gate="XOR", epochs=1000, learning_rate=0.0001, batch_size=64, k_epochs=64, epsilon=0.2, beta_kl=0.01, max_grad_norm=0.5):
    print(f"Training {gate} gate with {epochs} epochs, {learning_rate} learning rate, batch size {batch_size}, and KL beta {beta_kl}.")
    #Initialize Agent's Policy, Environment, parameter optimizer, and Total Correct Counter
    env = LogicGateEnv(gate)
    Policy_New = LogicNet()
    optimizer = optim.Adam(Policy_New.parameters(), lr=learning_rate)
    num_correct = 0.0

    for epoch in range(epochs):
        rewards_batch = []
        inputs_batch = []
        targets_batch = []

        # --- 1. Collect a Batch of Experiences ---
        # Loop agent prediction, recording important values to lists:
        for i in range(batch_size):
            # Get model inputs and target
            idx = random.randint(0, 3)
            inputs = env.data[idx]
            target = env.targets[idx]

            # Get model prediction
            # Get logits from current policy and formulate the model's prediction for reward calculation
            with torch.no_grad(): # No need to track gradients during data collection
                prediction_logits = Policy_New(inputs)
                # print(f"prediction logits: {prediction_logits}")
                pred = torch.round(torch.sigmoid(prediction_logits)).float()

            # Calculate reward
            reward = env.step(idx, pred)

            # Append to lists
            inputs_batch.append(inputs)
            rewards_batch.append(reward)
            targets_batch.append(target)

        # Convert collected batch lists into PyTorch tensors
        inputs_batch_tensor = torch.stack(inputs_batch)
        targets_batch_tensor = torch.stack(targets_batch)
        rewards_batch_tensor = torch.tensor(rewards_batch, dtype=torch.float32)

        num_correct += (rewards_batch_tensor).sum().item()  ### need to change
        # print(f"Number correct, this iteration: {(rewards_batch_tensor).sum().item()}")

        # Unsqueeze to ensure rewards_batch_t has the same shape as targets_batch_t for element-wise ops SHAPE:(1, batch_size)
        rewards_batch_t = rewards_batch_tensor.unsqueeze(1)

        # --- START OF ADVANTAGE CALCULATION ---
        # Calculate the mean of the rewards in the current batch
        mean_reward = rewards_batch_tensor.mean()

        # Calculate the standard deviation of the rewards in the current batch
        # Add a small epsilon (1e-8) to prevent division by zero in case all rewards are identical
        std_reward = rewards_batch_tensor.std() + 1e-8
        # print(f"rewards_batch_t shape: {rewards_batch_t.shape} || mean_reward: {mean_reward}")
        # Calculate the advantage for each time step in the batch using your specified formula
        advantages_of_batch = (rewards_batch_t - mean_reward) / (std_reward)
        # --- END OF ADVANTAGE CALCULATION ---

        # --- 2. Store "Old Policy" Parameters ---
        # Transfer the weights to the Old Policy Model
        Policy_Old = LogicNet()
        Policy_Old.load_state_dict(Policy_New.state_dict())
        Policy_Old.eval()       # Tells Pytorch not to calculate gradients for this network

        # Get log_probabilities for the collected 'targets' from the OLD policy
        # Detach these to prevent gradients from flowing back into old_net
        with torch.no_grad():
            old_logits = Policy_Old(inputs_batch_tensor)
            # Use the get_action_and_or_log_prob helper
            log_prob_old = Policy_Old.get_action_and_or_log_prob(inputs_batch_tensor, targets_batch_tensor).detach()
            # The .detach() is critical here to ensure old_net remains fixed.

        # --- 3. Inner Loop (K_epochs) --- GRPO Optimization iteration
        for _ in tqdm(range(k_epochs), desc=f"Epoch {epoch+1}/{epochs} (Inner K-Epochs)", leave=False):
            new_policy_logits = Policy_New(inputs_batch_tensor)
            log_prob_new = Policy_New.get_action_and_or_log_prob(inputs_batch_tensor, targets_batch_tensor)


            # --- KL Divergence Calculation ---
            # Create Bernoulli distributions for new and old policies using their logits
            p_dist = torch.distributions.Bernoulli(logits=new_policy_logits)
            q_dist = torch.distributions.Bernoulli(logits=old_logits) # Use the detached old_logits

            # Calculate KL divergence per sample, then take the mean over the batch
            kl_div_per_sample = torch.distributions.kl.kl_divergence(p_dist, q_dist)
            kl_loss = kl_div_per_sample.mean() # Mean over the batch


            # print(f"log_prob_new: {log_prob_new}")
            # print(f"log_prob_old: {log_prob_old}")

            # Calculate the ratio of each Trajectory in the Group
            # r_t(0) = π_0(a_t|s_t) / π_0_old(a_t|s_t) = exp(log(π_0(a_t|s_t) - log(π_0_old(a_t|s_t)))
            ratio = torch.exp(log_prob_new - log_prob_old)

            # print(f"Ratio: {ratio}")

            surrogate_1 = ratio * advantages_of_batch
            surrogate_2 = torch.clamp(input=ratio, min= 1.0 - epsilon, max= 1.0 + epsilon) * advantages_of_batch

            # Combine clipped loss with KL penalty
            # Remember: we minimize the negative of the main objective, and add the KL term
            # Maximize: min(...) - beta * D_KL(...) => Minimize: -min(...) + beta * D_KL(...)
            policy_objective_term = -torch.min(surrogate_1, surrogate_2).mean()

            # print(f"policy_objective_term: {policy_objective_term}")
            
            loss = policy_objective_term + beta_kl * kl_loss # Add KL term with beta_kl weight

            # In GRPO, the objective function is typically designed to be maximized (e.g., maximizing the expected return). Since PyTorch optimizers are designed for minimization, the common practice is to minimize the negative of the objective function.

            # Update the New Policy Model
            optimizer.zero_grad()
            loss.backward()
            # --- ADDING GRADIENT CLIPPING HERE TO LIMIT PARAMETER UPDATES---
            # torch.nn.utils.clip_grad_norm_(Policy_New.parameters(), max_norm=max_grad_norm)
            # ----------------------------------
            optimizer.step()



        # LOG IF ENOUGH EPOCHS HAVE ELAPSED
        if epoch % 100 == 0:
            avg_reward = rewards_batch_tensor.mean().item()
            print(f"Epoch {epoch}: Loss = {loss.item()}, Avg Reward = {avg_reward:.4f}, Mean Advantage: {advantages_of_batch.mean().item()}")
            # Validation Step
            print("Validating the Model:")
            with torch.no_grad():
                for i in range(4):
                    logits = Policy_New(env.data[i])
                    pred = torch.round(torch.sigmoid(logits)).item()
                    print(f"Input: {env.data[i].tolist()}, Logits: {logits}, Prediction: {pred}, Actual: {env.targets[i].item()}")

    print("Training completed.\n")
    print(f"Number of correct predictions: {num_correct}/{epochs * batch_size}")
    print(f"Accuracy: {num_correct/(epochs * batch_size)}%")

    print("\nTesting Trained Model:")
    for i in range(4):
        logits = Policy_New(env.data[i])
        pred = torch.round(torch.sigmoid(logits)).item()
        print(f"Input: {env.data[i].tolist()}, Prediction: {pred}, Actual: {env.targets[i].item()}")


In [33]:

# Run training
train_logic_gate("AND")

Training AND gate with 1000 epochs, 0.0001 learning rate, batch size 64, and KL beta 0.01.


Epoch 0: Loss = -0.004044245462864637, Avg Reward = 0.7031, Mean Advantage: 1.4901161193847656e-08
Input: [0.0, 0.0], Logits: tensor([-0.1039]), Prediction: 0.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([-0.0842]), Prediction: 0.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([0.0999]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([0.0773]), Prediction: 1.0, Actual: 1.0


Epoch 100: Loss = -0.012214857153594494, Avg Reward = 0.2031, Mean Advantage: 7.450580596923828e-09
Input: [0.0, 0.0], Logits: tensor([1.7961]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([2.1376]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([2.9123]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([3.2758]), Prediction: 1.0, Actual: 1.0


Epoch 200: Loss = -0.023754172027111053, Avg Reward = 0.2031, Mean Advantage: 7.450580596923828e-09
Input: [0.0, 0.0], Logits: tensor([5.5384]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([7.8225]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([8.8384]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([11.1225]), Prediction: 1.0, Actual: 1.0


Epoch 300: Loss = -0.03921424597501755, Avg Reward = 0.2812, Mean Advantage: 7.450580596923828e-09
Input: [0.0, 0.0], Logits: tensor([10.9531]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([16.9080]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([18.3358]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([24.2907]), Prediction: 1.0, Actual: 1.0


...

Epoch 600: Loss = -0.074170783162117, Avg Reward = 0.3438, Mean Advantage: 5.587935447692871e-09
Input: [0.0, 0.0], Logits: tensor([37.3896]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([64.5873]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([67.0637]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([94.2615]), Prediction: 1.0, Actual: 1.0


Epoch 700: Loss = -0.0753750130534172, Avg Reward = 0.2969, Mean Advantage: -1.4901161193847656e-08
Input: [0.0, 0.0], Logits: tensor([49.3583]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([86.7033]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([89.4987]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([126.8437]), Prediction: 1.0, Actual: 1.0


Epoch 800: Loss = -0.0681488960981369, Avg Reward = 0.1875, Mean Advantage: -1.1175870895385742e-08
Input: [0.0, 0.0], Logits: tensor([62.5212]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([111.0903]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([114.1397]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([162.7088]), Prediction: 1.0, Actual: 1.0


Epoch 900: Loss = -0.07719399034976959, Avg Reward = 0.2500, Mean Advantage: -3.3527612686157227e-08
Input: [0.0, 0.0], Logits: tensor([76.7607]), Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Logits: tensor([137.3252]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Logits: tensor([140.6774]), Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Logits: tensor([201.2419]), Prediction: 1.0, Actual: 1.0


Training completed.

Number of correct predictions: 16368.0/64000
Accuracy: 0.25575%

Testing Trained Model:
Input: [0.0, 0.0], Prediction: 1.0, Actual: 0.0
Input: [0.0, 1.0], Prediction: 1.0, Actual: 0.0
Input: [1.0, 0.0], Prediction: 1.0, Actual: 0.0
Input: [1.0, 1.0], Prediction: 1.0, Actual: 1.0
